<a href="https://colab.research.google.com/github/ze-16/Conversational-AI-Agent/blob/main/Conversational_speech_training_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import os
import gdown
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import BertModel


In [23]:
# Coding the file path and allowing access through link
file_id = '1mC-mj6zuojm8jWRBH46Vj2Gno1ZsXlRF'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'Text_data.csv'

In [24]:
# If statement to make sure file is only downloaded once and not repeatedly
if not os.path.exists(output):
    print("Downloading file...")
    gdown.download(url, output, quiet=False)
else:
    print(f"File already exists.")

File already exists.


In [25]:
# Loading the goemotions_2 dataset for EDA
df = pd.read_csv(output)

In [26]:
# Loading the dimensions of the dataset
df.shape

(70000, 37)

In [27]:
# Loading the first 10 values of the dataset to explore what the data looks like
df.head(10)

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,We can hope,ee3o3ko,darkenseyreth,EdmontonOilers,t3_ag4r9j,t1_ee3mhad,1.547529e+09,62,False,0,...,0,0,1,0,0,0,0,0,0,0
1,Shhh don't give them the idea!,eebl3z7,BoinkBoinkEtAliae,MurderedByWords,t3_ah3o76,t1_eeb68lo,1.547777e+09,76,False,0,...,0,0,0,0,0,0,0,0,0,0
2,"Thank you so much, kind stranger. I really nee...",ed4fe9l,savageleaf,raisedbynarcissists,t3_abwh00,t1_ed4etbj,1.546482e+09,24,False,0,...,0,0,0,0,0,0,0,0,0,0
3,Ion know but it would be better for you to jus...,efavtdu,CADDiLLXC,darknet,t3_al4njw,t3_al4njw,1.548800e+09,62,False,0,...,0,0,0,0,0,0,0,0,0,1
4,I'm honestly surprised. We should have fallen ...,ee2imz2,CorporalThornberry,CollegeBasketball,t3_afxt6t,t1_ee22nyr,1.547497e+09,55,False,0,...,0,0,0,0,0,0,0,0,0,0
5,Jurisprudence fetishist gets off on technicality.,ednv5fy,100Gold,TopMindsOfReddit,t3_ae3p1m,t1_ednn3pz,1.547068e+09,12,False,0,...,0,0,0,0,0,0,0,0,0,1
6,One of my worst nightmares. Ugh.,ed60o6e,June1111,ExpectationVsReality,t3_ac7y4f,t3_ac7y4f,1.546541e+09,55,False,0,...,0,0,0,0,0,0,0,0,0,0
7,Is this real? 🤔,edjldcm,Uneekusername101,instant_regret,t3_adr232,t3_adr232,1.546928e+09,72,False,0,...,0,0,0,0,0,0,0,0,0,0
8,My favorite pod when they are fresh & clear. S...,ee7j34k,_BAMBOOZLE_,juul,t3_agn37x,t3_agn37x,1.547662e+09,12,False,0,...,0,0,0,0,0,0,0,0,0,1
9,I get wound up over completely imagined conver...,eeztbjr,TheLiquidSquid,aspergers,t3_ajpzf0,t1_eeypbzh,1.548483e+09,61,False,0,...,0,0,1,0,1,0,0,0,0,0


In [28]:
# Check the names of the headings and explore their data-types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   text                  70000 non-null  object 
 1   id                    70000 non-null  object 
 2   author                70000 non-null  object 
 3   subreddit             70000 non-null  object 
 4   link_id               70000 non-null  object 
 5   parent_id             70000 non-null  object 
 6   created_utc           70000 non-null  float64
 7   rater_id              70000 non-null  int64  
 8   example_very_unclear  70000 non-null  bool   
 9   admiration            70000 non-null  int64  
 10  amusement             70000 non-null  int64  
 11  anger                 70000 non-null  int64  
 12  annoyance             70000 non-null  int64  
 13  approval              70000 non-null  int64  
 14  caring                70000 non-null  int64  
 15  confusion          

In [29]:
# Basic descriptive statistics for categorical variables
df.describe(include='object')

,text,id,author,subreddit,link_id,parent_id
count,70000,70000,70000,70000,70000,70000
unique,43293,43448,37837,483,34995,41480
top,Thank you.,ef6zc9b,[deleted],cringe,t3_afp0p6,t3_ae6ejj
freq,18,5,1236,295,35,25


In [30]:
# Checking for any missing values and their sum for each column
df.isna().sum()

,0
text,0
id,0
author,0
subreddit,0
link_id,0
parent_id,0
created_utc,0
rater_id,0
example_very_unclear,0
admiration,0


In [31]:
# Removing variables/columns that will be of no consequence to the result upon initial look
df = df.drop(columns=(['id','author','subreddit','link_id','parent_id','created_utc','rater_id','example_very_unclear']))

In [32]:
# Running a check to make sure the targetted columns have been removed
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            70000 non-null  object
 1   admiration      70000 non-null  int64 
 2   amusement       70000 non-null  int64 
 3   anger           70000 non-null  int64 
 4   annoyance       70000 non-null  int64 
 5   approval        70000 non-null  int64 
 6   caring          70000 non-null  int64 
 7   confusion       70000 non-null  int64 
 8   curiosity       70000 non-null  int64 
 9   desire          70000 non-null  int64 
 10  disappointment  70000 non-null  int64 
 11  disapproval     70000 non-null  int64 
 12  disgust         70000 non-null  int64 
 13  embarrassment   70000 non-null  int64 
 14  excitement      70000 non-null  int64 
 15  fear            70000 non-null  int64 
 16  gratitude       70000 non-null  int64 
 17  grief           70000 non-null  int64 
 18  joy   

In [33]:
# Upon further reflection, dictionary created to condense the range of emotions, for the sake of simplicity of providing feedback at project end
emotion_groups = {
    'anger': ['anger', 'annoyance', 'disapproval', 'disgust', 'fear', 'nervousness'],
    'joy': ['joy', 'admiration', 'amusement', 'approval', 'caring', 'desire',
            'excitement', 'gratitude', 'love', 'optimism', 'pride', 'relief'],
    'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
    'neutral': ['neutral', 'confusion', 'curiosity', 'realization', 'surprise']
}

In [34]:
# Function that helps map the 28 emotions into 4 core groups, in order of priority
def get_core_emotion(row):
  for col in emotion_groups['anger']:
        if row[col] == 1:
            return 'anger'

  for col in emotion_groups['sadness']:
        if row[col] == 1:
            return 'sadness'

  for col in emotion_groups['joy']:
        if row[col] == 1:
            return 'joy'

  return 'neutral'

In [35]:
# Applying the above function onto the dataset
df['core_emotion'] = df.apply(get_core_emotion, axis=1)

In [36]:
# Checking to see the distribution of each category
df['core_emotion'].value_counts()

,count
core_emotion,
neutral,26126
joy,25520
anger,12900
sadness,5454


In [37]:
# Writing the new organised emotions into a new csv file
df[['text', 'core_emotion']].to_csv('Text_data2.csv', index=False)

In [38]:
# Loading the new file and checking if the mapping worked
df = pd.read_csv('Text_data2.csv')
df.head()

,text,core_emotion
0,We can hope,joy
1,Shhh don't give them the idea!,anger
2,"Thank you so much, kind stranger. I really nee...",joy
3,Ion know but it would be better for you to jus...,neutral
4,I'm honestly surprised. We should have fallen ...,joy


In [39]:
# Checking to see the distribution of each category
df['core_emotion'].value_counts()

,count
core_emotion,
neutral,26126
joy,25520
anger,12900
sadness,5454


In [40]:
# Loading a pre-trained bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [42]:
# Creating a class to tokenize dataset for BERT
class TokenizetoBERT(Dataset):
  # Storing the dataset and the tokenizer into the class
    def __init__(self, dataframe, tokenizer, max_len=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
  # Gets the size of the data
    def __len__(self):
      return len(self.data)
  # Getting the text and labels and tokenizing them
    def __getitem__(self, index):
      # Getting the text
      text = str(self.data.iloc[index]['text'])
      # Getting the labels for the text
      labels = self.data.iloc[index]['core_emotion']
      # Mapping the strings to integers for computer interpretability
      label_map = {'neutral': 0, 'joy': 1, 'sadness': 2, 'anger': 3}
      label = label_map.get(labels, 0)
      # Tokenizing
      tokenizing = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

      return {
            'text': text,
            'input_ids': tokenizing['input_ids'].flatten(),
            'attention_mask': tokenizing['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [44]:
# Splitting the data based on variable 'core_emotion'
train_set,test_set = train_test_split(df,test_size=0.3, random_state=14,stratify=df['core_emotion'])

In [45]:
# Creating instances of the previously defined class
train_dataset = TokenizetoBERT(train_set, tokenizer)
test_dataset = TokenizetoBERT(test_set, tokenizer)

In [46]:
# Using the built in DataLoader class from pytorch
train_loader=DataLoader(train_dataset,batch_size=20,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=20,shuffle=True)